# INFO 607.  Tensorflow Project.  Exploratory Data Analysis
## May 10, 2020

In [1]:
import requests
import json
from time import sleep
import os
from os import path
from datetime import datetime, timedelta
from glob import glob
import re
import csv


import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np


### Helper Methods from the Quickstart

In [2]:
def read_data_json(typename, api, body):
    """
    read_data_json directly accesses the C3.ai COVID-19 Data Lake APIs using the requests library, 
    and returns the response as a JSON, raising an error if the call fails for any reason.
    ------
    typename: The type you want to access, i.e. 'OutbreakLocation', 'LineListRecord', 'BiblioEntry', etc.
    api: The API you want to access, either 'fetch' or 'evalmetrics'.
    body: The spec you want to pass. For examples, see the API documentation.
    """
    response = requests.post(
        "https://api.c3.ai/covid/api/1/" + typename + "/" + api, 
        json = body, 
        headers = {
            'Accept' : 'application/json', 
            'Content-Type' : 'application/json'
        }
    )
    response.raise_for_status()
    
    return response.json()

def fetch(typename, body, get_all = False, remove_meta = True):
    """
    fetch accesses the Data Lake using read_data_json, and converts the response into a Pandas dataframe. 
    fetch is used for all non-timeseries data in the Data Lake, and will call read_data as many times 
    as required to access all of the relevant data for a given typename and body.
    ------
    typename: The type you want to access, i.e. 'OutbreakLocation', 'LineListRecord', 'BiblioEntry', etc.
    body: The spec you want to pass. For examples, see the API documentation.
    get_all: If True, get all records and ignore any limit argument passed in the body. If False, use the limit argument passed in the body. The default is False.
    remove_meta: If True, remove metadata about each record. If False, include it. The default is True.
    """
    if get_all:
        has_more = True
        offset = 0
        limit = 2000
        df = pd.DataFrame()

        while has_more:
            body['spec'].update(limit = limit, offset = offset)
            response_json = read_data_json(typename, 'fetch', body)
            new_df = pd.json_normalize(response_json['objs'])
            df = df.append(new_df)
            has_more = response_json['hasMore']
            offset += limit
            
    else:
        response_json = read_data_json(typename, 'fetch', body)
        df = pd.json_normalize(response_json['objs'])
        
    if remove_meta:
        df = df.drop(columns = [c for c in df.columns if ('meta' in c) | ('version' in c)])
    
    return df
    
def evalmetrics(typename, body, remove_meta = True):
    """
    evalmetrics accesses the Data Lake using read_data_json, and converts the response into a Pandas dataframe.
    evalmetrics is used for all timeseries data in the Data Lake.
    ------
    typename: The type you want to access, i.e. 'OutbreakLocation', 'LineListRecord', 'BiblioEntry', etc.
    body: The spec you want to pass. For examples, see the API documentation.
    remove_meta: If True, remove metadata about each record. If False, include it. The default is True.
    """
    response_json = read_data_json(typename, 'evalmetrics', body)
    df = pd.json_normalize(response_json['result'])
    
    # get the useful data out
    df = df.apply(pd.Series.explode)
    if remove_meta:
        df = df.filter(regex = 'dates|data|missing')
    
    # only keep one date column
    date_cols = [col for col in df.columns if 'dates' in col]
    keep_cols =  date_cols[:1] + [col for col in df.columns if 'dates' not in col]
    df = df.filter(items = keep_cols).rename(columns = {date_cols[0] : "dates"})
    df["dates"] = pd.to_datetime(df["dates"])
    
    return df

#### Streamlined request for single item

In [3]:
def fetch_one(typename: str, body: dict, objs_only=True) -> dict:
    """
    Returns JSON output from single API call
    
    Args:
        typename: the C3.ai type name
        body: the body of the request
        objs_only: if True, remove the metadata and just returns the objects
        
    Returns:
        JSON response as dictionary
    
    """

    response = read_data_json(typename, 'fetch', body)
    if objs_only:
        for r in response['objs']:
            if 'meta' in r.keys():
                del(r['meta'])
                
        return response['objs']
    
    return response
    

### Load the location codes for the US into a Pandas DataFrame

In [4]:
def get_us_locations(file_name='C3-ai-Location-IDs.xlsx'): 
    """ Loads all US counties from C3 ai spreadsheet 
    
    Args:
        file_name: the name of the spreadsheet
        
    Returns:
        Pandas dataframe with the results
    
    """
                     
    locations = pd.read_excel(path.join('.', file_name), sheet_name='County IDs', header=2)
    us_locations = locations[locations.Country=='United States']
    
    return us_locations
    

### Get the basic population data for each of the 3429 counties

In [10]:
def make_outbreaklocation_body(county_id: str) -> dict:
    """ Forms the request body for a count for the outbreak location API 
    
    Args:
        count_id: the ID for the County
    
    Returns:
        The request body
    
    """
    return {
              "spec": {
                "filter": f"id == '{county_id}'"
              }
}

# fetch_one('outbreaklocation', make_outbreaklocation_body('Autauga_Alabama_UnitedStates'))


In [15]:
def load_population_data(file_name='counties.json'):
    """ Loads all population data for US counties and stores in a file called counties.json"""

    us_locations = get_us_locations()
    keep_going = True
    tries = 0
    while keep_going:
        try:
            with open(file_name) as file:
                county_data = json.load(file)
        except:    
            county_data = {}
        i = 0
        for county in us_locations['County id']:
            if county not in county_data.keys():
                try:
                    data = fetch_one('outbreaklocation',  make_outbreaklocation_body(county))
                    county_data[county] = data[0]
                    i += 1
                    if i % 100 == 0:
                        print(f'Saving: {i}')
                        with open(file_name, 'w') as file:
                            json.dump(county_data, file)
                except:
                    county_data[county] = None
                    print(f'Problem with {county}')
                sleep(1)
        with open('counties.json', 'w') as file:
            json.dump(county_data, file)
        if len(county_data) >= len(us_locations) or tries >= 5:
            keep_going = False
        else:
            tries += 1
        
def get_counties_df(file_name='counties.json'):
    with open(file_name) as file:
                county_data = json.load(file)
    
    df = pd.DataFrame.from_dict(county_data)
    
    data = [df[col] for col in df.columns]    
    
    # pivot
    return pd.DataFrame(data,columns=df.index, index=df.columns)
    
# get_counties_df()    


### Get the County Stats from the Census Bureau

In [12]:
def get_county_stats_df(file_name='county_stats.csv'):
    """ Get county land area (LND110210) by FIPS code """
    return pd.read_csv(path.join('.',file_name))[['fips','LND110210']]

get_county_stats_df()

# Equivalent data https://www.kaggle.com/benhamner/2016-us-election

,fips,LND110210
0,0,3531905.43
1,1000,50645.33
2,1001,594.44
3,1003,1589.78
4,1005,884.88
...,...,...
3190,56037,10426.65
3191,56039,3995.38
3192,56041,2081.26
3193,56043,2238.55


### Functions to Download the Evalmetrics Data

In [28]:

def get_last_file_date(county: str) -> str:
    max_date = '2020-01-01'
    files = glob(path.join('.', 'data', f'{county}*.psv'))
    if not files:
        return max_date
    for file in files:
        match = re.search(r'(\d\d\d\d-\d\d-\d\d).psv', file)
        if match:
            max_date = max(max_date, match.group(1))
    return max_date
            
    
def download_evalmetrics_data():
    """ Downloads the evalmetrics data from the last download through current """
    
    today = datetime.now().strftime('%Y-%m-%d')

    # Get the list of counties
    counties_file = path.join('.', 'counties.json')
    with open(counties_file) as file:
        counties = json.load(file)

    # Iterate through counties saving the time series data
    for county, details in counties.items():
        print(county)

        # Skip if missing county details
        if not details:
            continue

        # Get the last date we processed
        last_date = get_last_file_date(county)

        if last_date == today:
            continue

        # Get the data for the county from the last date processed
        body = {"spec" : {
                            "ids" : [county],
                            "expressions": [ "JHU_ConfirmedCases", "JHU_ConfirmedDeaths", "JHU_ConfirmedRecoveries"], 
                            "start" : last_date,
                            "end" : today,
                            "interval" : "DAY",
                        }
                }

        try:
            df = evalmetrics("outbreaklocation", body)
            file_name = path.join('.', 'data', f'{county}-{last_date}-{today}.psv')
            df.to_csv(file_name, sep='|')
        except Exception as e:
            print(f'Error processing {county}: {e}')
        
        sleep(1)
  
# download_evalmetrics_data()    

### Load all the downloaded data and produce raw DataFrame

In [25]:
def append_to_dataframe(files):
    name_pattern = re.compile('([\w_]+)')
    county_df = None

    for file in files:
        base = path.basename(file)
        try:
            county = name_pattern.match(base).group(1)
            records = []
            with open(file) as fd:
                reader = csv.reader(fd, delimiter='|')
                # advance past the header
                next(iter(reader))
                records = [[county, row[1], row[2], row[4], row[6]] for row in reader]
            temp_df = pd.DataFrame(records, columns=['county', 'date', 'confirmed_cases', 'confirmed_deaths', 'confirmed_recoveries'])
            if county_df is None:
                county_df = temp_df
            else:
                county_df = pd.concat([county_df, temp_df], ignore_index=True)
        except Exception as e:
            print(f'Failure on file {base} with error {e}')
        
    return county_df

def join_evalmetrics_to_county(evalmetrics_df, counties_df, county_stats_df):
    df = evalmetrics_df.merge(counties_df, how='left', left_on='county', right_index=True, sort=True)
    # df.fips = [int(fips['id']) for fips in df.fips]
    fips = []
    for f in df.fips:
        if isinstance(f, dict):
            fips.append(int(f['id']))
        else:
            fips.append(f)
    df.fips = fips
    df = df.merge(county_stats_df, how='left', left_on='fips', right_on='fips')
    df = df.drop(columns=['location', 'id', 'name', 'version', 'typeIdent'])
    
    return df.sort_values(by=['date', 'county'])



### Save the raw dataframe to a file

In [30]:
def save_raw_df(df: pd.DataFrame):
    df.to_pickle(path.join('.', 'raw_evalmetrics_df.pkl'))

### Runners
#### The following runners can take a significant amount of time to process

##### Download Evalmetrics Data

In [ ]:
download_evalmetrics_data()

#### Create the raw dataframe

In [29]:
evalmetric_df = append_to_dataframe([path.join('.','data', file) for file in os.listdir(path.join('.', 'data'))])

counties_df = get_counties_df()    
county_stats_df = get_county_stats_df()

df = join_evalmetrics_to_county(evalmetric_df, counties_df, county_stats_df)
save_raw_df(df)
df

Failure on file .ipynb_checkpoints with error 'NoneType' object has no attribute 'group'


,county,date,confirmed_cases,confirmed_deaths,confirmed_recoveries,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,fips,LND110210
0,Abbeville_SouthCarolina_UnitedStates,2020-01-01,0.0,0.0,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
144,Acadia_Louisiana_UnitedStates,2020-01-01,0.0,0.0,0.0,7.0,171.0,186.0,62045.0,22001.0,655.12
288,Accomack_Virginia_UnitedStates,2020-01-01,0.0,0.0,0.0,10.0,46.0,52.0,32316.0,51001.0,449.50
432,Ada_Idaho_UnitedStates,2020-01-01,0.0,0.0,0.0,97.0,1195.0,1305.0,481587.0,16001.0,1052.58
576,Adair_Iowa_UnitedStates,2020-01-01,0.0,0.0,0.0,4.0,25.0,25.0,7152.0,19001.0,569.27
...,...,...,...,...,...,...,...,...,...,...,...
466703,Yuma_Arizona_UnitedStates,2020-05-23,508.0,7.0,0.0,42.0,447.0,456.0,213787.0,4027.0,5513.99
466847,Yuma_Colorado_UnitedStates,2020-05-23,12.0,0.0,0.0,6.0,30.0,30.0,10019.0,8125.0,2364.41
466991,Zapata_Texas_UnitedStates,2020-05-23,8.0,0.0,0.0,NaN,NaN,NaN,14179.0,48505.0,998.41
467135,Zavala_Texas_UnitedStates,2020-05-23,11.0,0.0,0.0,NaN,NaN,NaN,11840.0,48507.0,1297.41


In [ ]:
#### To be used for small testing
# evalmetrics_df = append_to_dataframe([path.join('.','data','Wayne_Ohio_UnitedStates-2020-01-01-2020-05-14.psv'),
#                      path.join('.','data','Faulkner_Arkansas_UnitedStates-2020-01-01-2020-05-14.psv')])